In [1]:
from carbon import CarbonSimulatorUI, CarbonOrderUI, P, __version__, __date__
from math import sqrt
import numpy as np
from matplotlib import pyplot as plt
print(f"Carbon v{__version__} ({__date__})")
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSimulatorUI))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonOrderUI))

Carbon v2.2-BETA5 (10/Jan/2022)
CarbonSimulatorUI v2.4-beta1 (12/Jan/2023)
CarbonOrderUI v1.5.1 (07/Jan/2023)


# Carbon Simulation - Test 49 - Limit Prices

NBTEST: NOTEST_DEFAULT = TEST

## Limit prices

In [2]:
SimR = CarbonSimulatorUI(pair="ETH/USDC", raiseonerror=True)
SimNR = CarbonSimulatorUI(pair="ETH/USDC", raiseonerror=False)
SimR, SimNR

(CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETH/USDC', mm='exact', xf=True),
 CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETH/USDC', mm='exact', xf=True))

In [3]:
SimR.add_order("ETH", 10, 2000, 3000)
SimNR.add_order("ETH", 10, 2000, 3000)
SimR.add_order("USDC", 10000, 1000, 500)
SimNR.add_order("USDC", 10000, 1000, 500)
assert len(SimR.state()["orders"]) == 4
assert len(SimNR.state()["orders"]) == 4

In [4]:
help(SimR._trade)

Help on method _trade in module carbon.simulators.carbon_simulator:

_trade(tkn: str, amt: Any, support_partial: bool, carbon_pair: carbon.pair.CarbonPair, match_by: str = None, trade_description: str = None, execute: bool = True, limit_price: Any = None, limit_amt: Any = None, inpair: bool = True, use_positions: List[int] = None, threshold_orders: int = None, use_positions_matchlevel: List[int] = [], is_by_target: bool = False) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    PRIVATE - executes a trade
    
    :tkn:                   the token that is being SOLD by the AMM, eg "ETH"*
    :amt:                   the amount to be traded*
    :carbon_pair:           the CarbonPair class of the token pair
    :match_by:              either MATCH_BY_SOURCE or MATCH_BY_TARGET
    :trade_description:     human-readable description of the trade, eg "amm vs trader, buy vs sell, for amt"
    :execute:               if True (default), the trade is ex

In [5]:
assert SimR.trader_buys == SimR.amm_sells
assert SimR.trader_sells == SimR.amm_buys
assert SimR.trader_buys != SimR.amm_buys
assert SimR.trader_sells != SimR.amm_sells
help(SimR.trader_buys)
help(SimR.trader_sells)

Help on method amm_sells in module carbon.simulators.carbon_simulator:

amm_sells(tkn: str, amt: Any, pair: str = None, execute: bool = True, inpair: bool = True, limit_price: Any = None, limit_amt: Any = None, threshold_orders: int = 10, use_positions: List[int] = None, use_positions_matchlevel: List[int] = [], support_partial: bool = False) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    the AMM sells (and the trader buys) `amt` > 0 of `tkn`
    
    :tkn:               the token sold by the AMM and bought by the trader, eg "ETH"
    :amt:               the amount sold by the AMM and bought by the trader (must be positive)
    :pair:              the token pair to which the trade corresponds, eg "ETHUSD"
    :execute:           if True (default), the trade is executed; otherwise only routing is shown
    :inpair:            if True, only match within pair; if False (default), route through all available pairs
    :limit_price:       the l

In [6]:
r = SimR.amm_sells("ETH", 1, execute=False)
assert r["success"] == True
r["trades"]

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,False,False,None,1.0,ETH,2037.386743,USDC,ETHUSDC,0,1,2037.386743,USDC per ETH
0,0,0,A,AMM sells 1ETH buys 2037USDC,True,False,True,None,1.0,ETH,2037.386743,USDC,ETHUSDC,[0],1,2037.386743,USDC per ETH


### Check that you can't give both price and amount, both by amount and by target

In [7]:
try:
    SimR.amm_sells("ETH", 1, limit_price=1000, limit_amt=1000)
    raise RuntimeError("should raise")
except ValueError as e:
    print(e)

('Not both limit_amt and limit_price can be given', 1000, 1000)


In [8]:
try:
    SimR.amm_buys("ETH", 1, limit_price=1000, limit_amt=1000)
    raise RuntimeError("should raise")
except ValueError as e:
    print(e)

('Not both limit_amt and limit_price can be given', 1000, 1000)


In [9]:
try:
    SimR.amm_sells("USDC", 1, limit_price=1000, limit_amt=1000)
    raise RuntimeError("should raise")
except ValueError as e:
    print(e)

('Not both limit_amt and limit_price can be given', 1000, 1000)


In [10]:
try:
    SimR.amm_buys("USDC", 1, limit_price=1000, limit_amt=1000)
    raise RuntimeError("should raise")
except ValueError as e:
    print(e)

('Not both limit_amt and limit_price can be given', 1000, 1000)


In [11]:
assert SimNR.amm_sells("ETH", 1, limit_price=1000, limit_amt=1000)["success"] == False
assert SimNR.amm_buys("ETH", 1, limit_price=1000, limit_amt=1000)["success"] == False
assert SimNR.amm_sells("USDC", 1, limit_price=1000, limit_amt=1000)["success"] == False
assert SimNR.amm_buys("USDC", 1, limit_price=1000, limit_amt=1000)["success"] == False

### Check price limits

In [12]:
r = SimR.trader_buys("ETH", 1, execute=False)
price_buy_eth = r["trades"]["price"].iloc[0]
assert int(price_buy_eth) == 2037

In [13]:
r = SimR.trader_buys("USDC", 1000, execute=False)
price_buy_usdc = r["trades"]["price"].iloc[0]
assert int(price_buy_usdc) == 970

In [14]:
r = SimR.trader_sells("ETH", 1, execute=False)
price_sell_eth = r["trades"]["price"].iloc[0]
assert int(price_sell_eth) == 971

In [15]:
r = SimR.trader_sells("USDC", 1000, execute=False)
price_sell_usdc = r["trades"]["price"].iloc[0]
assert int(price_sell_usdc) == 2018

In [16]:
print(f"buy  ETH   @ {price_buy_eth:10.0f} USDC/ETH")
print(f"buy  USDC  @ {price_buy_usdc:10.0f} USDC/ETH")
print(f"sell ETH   @ {price_sell_eth:10.0f} USDC/ETH")
print(f"sell USDC  @ {price_sell_usdc:10.0f} USDC/ETH")

buy  ETH   @       2037 USDC/ETH
buy  USDC  @        971 USDC/ETH
sell ETH   @        972 USDC/ETH
sell USDC  @       2018 USDC/ETH


In [17]:
limitfail = lambda  r: "fail" if r["trades"].query("aggr==True")["limitfail"].iloc[0] else "nofail"

worse than limit transactions

In [18]:
fail_r = (
    limitfail(SimR.trader_buys("ETH", 1, limit_price = int(price_buy_eth) - 1, execute=False)),
    limitfail(SimR.trader_buys("USDC", 1000, limit_price = int(price_buy_usdc) + 1, execute=False)),
    limitfail(SimR.trader_sells("ETH", 1, limit_price = int(price_sell_eth) + 1, execute=False)),
    limitfail(SimR.trader_sells("USDC", 1000, limit_price = int(price_sell_usdc) - 1, execute=False)),
)
assert fail_r == ("fail",)*4
fail_r

[_trade] limit_price=2036, price_avg=2037.386743337496
[_trade] limit_price=971, price_avg=970.7106781187076
[_trade] limit_price=972, price_avg=971.544131218801
[_trade] limit_price=2017, price_avg=2018.3503419082042


('fail', 'fail', 'fail', 'fail')

better than limit transactions

In [19]:
fail_r = (
    limitfail(SimR.trader_buys("ETH", 1, limit_price = int(price_buy_eth) + 1, execute=False)),
    limitfail(SimR.trader_buys("USDC", 1000, limit_price = int(price_buy_usdc) - 1, execute=False)),
    limitfail(SimR.trader_sells("ETH", 1, limit_price = int(price_sell_eth) - 1, execute=False)),
    limitfail(SimR.trader_sells("USDC", 1000, limit_price = int(price_sell_usdc) + 1, execute=False)),
)
assert fail_r == ("nofail",)*4
fail_r

[_trade] limit_price=2038, price_avg=2037.386743337496
[_trade] limit_price=969, price_avg=970.7106781187076
[_trade] limit_price=970, price_avg=971.544131218801
[_trade] limit_price=2019, price_avg=2018.3503419082042


('nofail', 'nofail', 'nofail', 'nofail')

### Check amount limits

worse than limit transactions

In [20]:
fail_r = (
    limitfail(SimR.trader_buys("ETH", 1, limit_amt = (int(price_buy_eth) - 1)*1, execute=False)),
    limitfail(SimR.trader_buys("USDC", 1000, limit_amt = 1000/(int(price_buy_usdc) + 1), execute=False)),
    limitfail(SimR.trader_sells("ETH", 1, limit_amt = (int(price_sell_eth) + 5)*1, execute=False)),
    limitfail(SimR.trader_sells("USDC", 1000, limit_amt = 1000/(int(price_sell_usdc) - 1), execute=False)),
)
assert fail_r == ("fail",)*4
fail_r

[_trade] limit_price=2036.0000000000002, price_avg=2037.386743337496
[_trade] limit_price=971.0, price_avg=970.7106781187076
[_trade] limit_price=976.0, price_avg=971.544131218801
[_trade] limit_price=2017.0, price_avg=2018.3503419082042


('fail', 'fail', 'fail', 'fail')

In [21]:
price_sell_usdc

2018.3503419082042

In [22]:
price_sell_usdc

2018.3503419082042

better than limit transactions

In [23]:
fail_r = (
    limitfail(SimR.trader_buys("ETH", 1, limit_amt = (int(price_buy_eth) + 1)*1, execute=False)),
    limitfail(SimR.trader_buys("USDC", 1000, limit_amt = 1000/(int(price_buy_usdc) - 1), execute=False)),
    limitfail(SimR.trader_sells("ETH", 1, limit_amt = (int(price_sell_eth) - 1)*1, execute=False)),
    limitfail(SimR.trader_sells("USDC", 1000, limit_amt = 1000/(int(price_sell_usdc) + 1), execute=False)),
)
assert fail_r == ("nofail",)*4
fail_r

[_trade] limit_price=2038.0000000000002, price_avg=2037.386743337496
[_trade] limit_price=968.9999999999999, price_avg=970.7106781187076
[_trade] limit_price=970.0, price_avg=971.544131218801
[_trade] limit_price=2019.0, price_avg=2018.3503419082042


('nofail', 'nofail', 'nofail', 'nofail')

### Check amount limits (ranges)

In [24]:
def goalseek(f, target, xlo, xhi):
    if not f(xhi)>target and f(xlo)<target:
        raise ValueError("must have f(xhi) > target, f(xlo)<target ", f(xhi), f(xlo))
    return _goalseek(f, target, xlo, xhi)
    
def _goalseek(f, target, xlo, xhi):
    xmid = 0.5*(xhi+xlo)
    #print(xlo, xhi, xmid)
    if abs(xhi-xlo)<1: return xmid
    if f(xmid) > target:
        return _goalseek(f, target, xlo, xmid)
    else:
        return _goalseek(f, target, xmid, xhi)
limitfail_ind = lambda  r: 0 if r["trades"].query("aggr==True")["limitfail"].iloc[0] else 1

In [25]:
x = goalseek(lambda x: limitfail_ind(SimR.trader_buys("ETH", 1, limit_amt=x, execute=False)), 0.5, 500, 3000)
assert int(x) == int(price_buy_eth)

[_trade] limit_price=3000.0, price_avg=2037.386743337496
[_trade] limit_price=1749.9999999999998, price_avg=2037.386743337496
[_trade] limit_price=2375.0, price_avg=2037.386743337496
[_trade] limit_price=2062.5, price_avg=2037.386743337496
[_trade] limit_price=1906.2499999999998, price_avg=2037.386743337496
[_trade] limit_price=1984.375, price_avg=2037.386743337496
[_trade] limit_price=2023.4375, price_avg=2037.386743337496
[_trade] limit_price=2042.9687499999998, price_avg=2037.386743337496
[_trade] limit_price=2033.203125, price_avg=2037.386743337496
[_trade] limit_price=2038.0859375000002, price_avg=2037.386743337496
[_trade] limit_price=2035.64453125, price_avg=2037.386743337496
[_trade] limit_price=2036.8652343750002, price_avg=2037.386743337496
[_trade] limit_price=2037.4755859375002, price_avg=2037.386743337496


2037.17041015625

In [27]:
x = goalseek(lambda x: limitfail_ind(SimR.trader_sells("ETH", 1, limit_amt=x, execute=False)), 0.5, 3000, 500)
assert int(x) == int(price_sell_eth)

[_trade] limit_price=500.0, price_avg=971.544131218801
[_trade] limit_price=1749.9999999999998, price_avg=971.544131218801
[_trade] limit_price=1125.0, price_avg=971.544131218801
[_trade] limit_price=812.5, price_avg=971.544131218801
[_trade] limit_price=968.75, price_avg=971.544131218801
[_trade] limit_price=1046.875, price_avg=971.544131218801
[_trade] limit_price=1007.8125, price_avg=971.544131218801
[_trade] limit_price=988.2812500000001, price_avg=971.544131218801
[_trade] limit_price=978.5156249999999, price_avg=971.544131218801
[_trade] limit_price=973.6328124999999, price_avg=971.544131218801
[_trade] limit_price=971.1914062499999, price_avg=971.544131218801
[_trade] limit_price=972.412109375, price_avg=971.544131218801
[_trade] limit_price=971.8017578125001, price_avg=971.544131218801
